# Sensitivity Analysis

In [2]:
import os
import pandas as pd
import pickle
import numpy as np

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
data_save_dir = "/cluster/home/kamara/syntax-shap/data"

model_name = "gpt2"
threshold_ = 0.5


In [10]:

folder_path = f"/cluster/work/zhang/kamara/syntax-shap/results_ids/scores/{model_name}"  # Replace with your folder path

# Initialize a list to store the dataframes
df_list = []

# Loop through each file in the folder
for root, dirs, files in os.walk(folder_path):
    for file in files:
        full_path = os.path.join(root, file)
        parts = file.split('_')
        if (file.endswith('.pkl')):
            # Extract dataset, model name, and algorithm from the filename
            dataset = parts[-5]
            model_name = parts[-4]
            algorithm = parts[-3]
            seed = parts[-2]
            threshold = parts[-1].split('.pkl')[0]  # Removes the file extension

            if eval(threshold) == threshold_:
                # Construct the full file path
                file_path = os.path.join(folder_path, file)

                # Read the pickle file
                with open(full_path, 'rb') as file:
                    data = pickle.load(file)

                # Assuming the content is a dictionary, create a DataFrame
                df = pd.DataFrame(data)
                # Add the extracted details as new columns
                df['algorithm'] = algorithm
                df['n_token'] = df['input'].str.split().apply(len)
                df['seed'] = seed
                df['dataset'] = dataset

                # Append to the list of dataframes
                df_list.append(df)

# Concatenate all dataframes in the list
df_final_single_threshold = pd.concat(df_list, ignore_index=True)
df_final_single_threshold


,fid_keep_rd,fid_keep,fid_rmv,log_odds_keep,acc_at_k,prob_diff_at_k,input_id,input,algorithm,n_token,seed,dataset
0,0.075490,0.044658,0.087215,-0.717511,0.5,0.372153,195,A rabbit walks with,partition,4,0,negation
1,0.008912,-0.002078,0.008912,0.209548,0.2,0.394719,323,Anna is a mother with a,partition,6,0,negation
2,0.013380,0.013380,0.003083,-9.501555,0.2,0.740757,531,A person has,partition,3,0,negation
3,0.141438,0.152168,0.152168,-11.932746,0.0,0.582714,234,A lion has no,partition,4,0,negation
4,0.035531,0.035531,0.029419,-10.478192,0.1,0.527202,495,A magazine has,partition,3,0,negation
...,...,...,...,...,...,...,...,...,...,...,...,...
20616,0.077730,0.068150,0.077730,-2.093478,0.3,0.707962,2619,Light can go around the earth three times in one,syntax,10,0,generics
20617,0.569296,0.723959,0.832887,-1.904326,0.5,0.592929,2841,Capital goods are goods which are necessary fo...,syntax,10,0,generics
20618,0.014829,0.023945,0.045868,-0.738185,0.7,0.045908,271,Thunder is made when lightning,syntax,5,0,generics
20619,0.021300,0.017070,0.021300,-1.616146,0.1,0.590119,505,More reactive acids are often more,syntax,6,0,generics


In [18]:
df = df_final_single_threshold.groupby(['dataset', 'algorithm','seed']).mean().reset_index()

In [19]:
#df = df[df['dataset'] == 'negation']
df.groupby(['dataset', 'algorithm'])['prob_diff_at_k'].agg(['mean', 'std'])

mean       std
dataset    algorithm                    
generics   lime       0.588423       NaN
           partition  0.549496       NaN
           shap       0.517883       NaN
           syntax     0.515036       NaN
           syntax-w   0.565909       NaN
negation   lime       0.522205  0.002869
           partition  0.458341  0.014372
           shap       0.485977  0.011432
           syntax     0.463532  0.010265
           syntax-w   0.457848  0.012522
rocstories lime       0.585379       NaN
           partition  0.537441       NaN
           shap       0.516739       NaN
           syntax     0.496689       NaN
           syntax-w   0.539308       NaN